# Reading the data and turning into a Dataframes

From the yelp dataset, the business and restaurants json files contain the data of interest. 
The data set is downloaded from https://www.yelp.com/dataset/challenge

In [1]:
import os # Miscellaneous operating system interfaces, used for finding path of files
import pandas as pd # for data manipulation and analysis.

In [2]:
path= os.path.abspath("business.json")
business = pd.read_json (path,lines=True)

In [3]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


Keep only businesses with 'Restaurants' in their category and 'Toronto' in city Keep only 'business_id' and 'categories' columns

In [4]:
restaurants= business[business['categories'].str.contains('Restaurants',case=False,na=False)]
restaurants= restaurants[restaurants['city'].str.contains('Toronto',case=False,na=False)]
restaurants=restaurants[['business_id','categories']]
restaurants.head()

,business_id,categories
29,NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas..."
32,SP_YXIEwkFPPl_9anCYmpQ,"Restaurants, Nightlife, Breakfast & Brunch, Ve..."
50,mlHC2XcU9Bows6cnYEmRgg,"Restaurants, Breakfast & Brunch, Bars, Modern ..."
83,RFbMVekR8lU9tPJ8sWrwHg,"Italian, Restaurants"
96,NPHZkn1e-tSJAbo8Zm9rYw,"Tex-Mex, Mexican, Restaurants"


Read the review json in chucks, otherwise memory error and merge the reviews with the restaurants defined above

In [5]:
path= os.path.abspath("review.json")
size=100000
review = pd.read_json (path,lines=True,chunksize=size)
chunk_list = []
n=True
for chunk_review in review:

        
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    chunk_merged = pd.merge(restaurants, chunk_review, on='business_id', how='inner')
    chunk_list.append(chunk_merged)
    
    
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
df.head()

,business_id,categories,user_id,review_stars,text,date
0,NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",fd25NWbvLdNFJV_MoAljIg,4,Pretty solid vegan spot. Had a rice bowl there...,2017-10-11 00:28:10
1,NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",AyxTBqPJjYqlY55vfNfB2Q,5,"What a great spot for a delicious, healthy lun...",2015-01-19 01:40:08
2,NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",-KvhDluq5GhtiafY4xK2Sw,2,I really do love Bolt Fresh Bar. I've gone the...,2018-05-30 14:04:50
3,NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",75DUiVsWkGZRW--zySVORA,4,"The vibe: Clean, bright vegetarian take out!\n...",2014-02-25 19:39:12
4,NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",z23KjDT_pUtORz4ptHnUAg,2,I went here once to get the protein bowl and i...,2017-05-27 00:14:14


save the dataframes for easy access later
restaurants includes business ID and categories
restaurant review contains business ID, categories, user_id, review_stars ,text and date.

In [6]:
df.to_pickle('resturantreviews')
restaurants.to_pickle('restaurants')